In [217]:
import numpy as np
import sys, os
import _pickle as pickle

import plotly.graph_objects as go
import plotly.express as ex
from cytoolz.itertoolz import concat, sliding_window
import scipy.signal as signal

sys.path.append("../")
import func as f

In [4]:
# Load data
data_path = "../../data/"
data = f.load(data_path+"ONE_R2-default-One.pbz2")
d1 = pickle.loads(data[0])  # only inspect the first animation sequence

In [63]:
# Extract the contact info per joint per frame
contact_info = []
for f in d1["frames"]:
    contacts = np.asarray([jo["contact"] for jo in f]).astype(np.int)
    contact_info.append(contacts)
contact_info = np.vstack(contact_info)
print(contact_info.shape)

(120, 21)


In [67]:
# plot the contact info
cInfo = contact_info.T
x = np.arange(contact_info.shape[0])

fig = go.Figure()
i = 0
for jo in cInfo:
    fig.add_trace(go.Scatter(x=x, y=jo, name="J"+str(i)))
    i+=1
fig.show()

In [186]:
def normalise(block_fn:np.ndarray, n_windows:int=10) -> np.ndarray:
    """
    Takes an ndarray of shape(n_joints, n_frames), where 1 = contact and 0 otherwise.
    :param block_fn:
    :param n_windows:
    :return:
    """
    window_size = block_fn.shape[1] / n_windows
    idx = (np.arange(n_windows) * window_size).astype(np.int)
    for i, ii in zip(idx[:-1], idx[1:]):
        slice = block_fn[:, i:ii]
        mean = np.mean(slice, axis=1, keepdims=True)
        std = np.std(slice, axis=1, keepdims=True)
        std[std == 0] = 1

        block_fn[:, i:ii] = (slice-mean) / std

    f_sample = block_fn.shape[1] / 60


    filter = signal.butter(3, 1/3.25, "low", analog=False, output="sos")
    block_fn = signal.sosfilt(filter, block_fn)
    return block_fn

def calc_local_motion_phase(block_fn:np.ndarray) -> np.ndarray:
    """
    Takes the contact info and approximates the local motion phase, as described in [Local Motion Phases for Learning Multi-Contact Character Movements]
    :param block_fn:
    :return:
    """
    block_fn = normalise(block_fn)
    # WIP....
    return block_fn



In [185]:
# plot the normalised contact info
cInfo = calc_local_motion_phase(np.copy(contact_info.T))
x = np.arange(contact_info.shape[0])

fig = go.Figure()
i = 0
for jo in cInfo:
    fig.add_trace(go.Scatter(x=x, y=jo, name="J"+str(i)))
    i+=1
fig.show()


In [204]:
# test evostrat....
# didnt work....
from evostra import EvolutionStrategy

Frames = 120
a = np.ones(Frames)
f = np.ones(Frames)
s = np.zeros(Frames)
b = np.zeros(Frames)
params = np.column_stack([a,f,s,b])
t = np.arange(Frames)

def infer(t, params):
    return params[0] * np.sin(params[1] * t - params[2]) + params[3]

def loss(params, row_fn, N=60):
    return np.sqrt(
        (infer(t, params) - row_fn)**2 / N
    )

def get_reward(params):
    global container
    container[0] = params
    return -loss(container[0], row_fn)

container = [params]
row_fn = cInfo[14]
es = EvolutionStrategy(container[0], get_reward, population_size=10, sigma=0.1, learning_rate=0.03, decay=0.995, num_threads=1)
es.run(100, print_step=20)

ValueError: operands could not be broadcast together with shapes (4,) (120,) 

In [210]:
from scipy.optimize import minimize

# test L-BFGS-B...
# and it works...maybe....

def func(x, *args):
    t = args[0]
    y = args[1]

    return np.sqrt(
        (infer(t, x) - y)**2 / 60
    )

results = []
for ts in t:
    res = minimize(func, params[ts], args=(ts, row_fn[ts]), method="L-BFGS-B")
    results.append(res)

In [215]:
# plot the approximated motion phases
amp = np.asarray([res.x[0] for res in results], dtype=np.float32)
freq = np.asarray([res.x[0] for res in results], dtype=np.float32)
phase = np.asarray([res.x[0] for res in results], dtype=np.float32)
bias = np.asarray([res.x[0] for res in results], dtype=np.float32)

x = np.arange(contact_info.shape[0])
y = infer(t, [amp, freq, phase, bias])

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
fig.show()


# Testing on another motion data


In [237]:
# testing one anohter motion data
data_path = "../../data/"
data = f.load(data_path+"ONE_R1-default-One.pbz2")
d1 = pickle.loads(data[0])

contact_info = []
for f in d1["frames"]:
    contacts = np.asarray([jo["contact"] for jo in f]).astype(np.int)
    contact_info.append(contacts)
contact_info = np.vstack(contact_info)
print(contact_info.shape)

cInfo = contact_info.T
x = np.arange(contact_info.shape[0])

fig = go.Figure()
i = 0
for jo in cInfo:
    fig.add_trace(go.Scatter(x=x, y=jo, name="J"+str(i)))
    i+=1
fig.show()


cInfo = calc_local_motion_phase(np.copy(contact_info.T))


x = np.arange(contact_info.shape[0])

fig = go.Figure()
i = 0
for jo in cInfo:
    fig.add_trace(go.Scatter(x=x, y=jo, name="J"+str(i)))
    i+=1
fig.show()



Frames = 120
a = np.ones(Frames)
f = np.ones(Frames)
s = np.ones(Frames)
b = np.ones(Frames)
params = np.column_stack([a,f,s,b])
t = np.arange(Frames)
row_fn = cInfo[6]

def infer(t, params):
    return params[0] * np.sin(params[1] * t - params[2]) + params[3]

from scipy.optimize import minimize

def func(x, *args):
    t = args[0]
    y = args[1]

    return np.sqrt(
        (infer(t, x) - y)**2 / 60
    )

results = []
for ts in t:
    res = minimize(func, params[ts], args=(ts, row_fn[ts]), method="L-BFGS-B")
    results.append(res)

amp = np.asarray([res.x[0] for res in results], dtype=np.float32)
freq = np.asarray([res.x[0] for res in results], dtype=np.float32)
phase = np.asarray([res.x[0] for res in results], dtype=np.float32)
bias = np.asarray([res.x[0] for res in results], dtype=np.float32)

x = np.arange(contact_info.shape[0])
y = infer(t, [amp, freq, phase, bias])

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
fig.show()

AttributeError: 'numpy.ndarray' object has no attribute 'load'